# Introduction à Unets
L'architecture Unet est un réseau de neurones initialement développé pour la segmentation d'images biomédicales à l'Université de Freiburg, en Allemagne.
L'Unet est une sorte d'autoencodeur mais avec des connections résiduelles entre l'encodeur et le décodeur afin d'éviter la disparition du gradient et de connecter les informations extraites par l'encodeur au décodeur plus facilement. Ces connections permettent au réseasu de capturer et de propager efficacement les informations de contexte.

![Unet architecture](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/TP4_Unet/u-net-architecture.png)


L'Unet est composée d'une structure encodeur-décodeur avec des connexions de saut (skip connections), lui permettant de capturer à la fois des features de haut et bas niveau.

L'architecture Unet en détail:

Encodeur : L'encodeur est responsable de l'extraction des features de l'image en entrée. Il consiste de plusieurs blocs de couche de convolution et un maxpooling entre chaque bloc pour réduire la taille de l'image.

Bridge: Le bridge sert à connecter les features extraits par l'encodeur à chaque niveau à l'entrée du décodeur, en concatenant la sortie de chaque bloc de l'encodeur à la sortie du bloc correspondant du décodeur.

Décodeur : Le décodeur s'occupe de la tâche essentielle de remapper les features de haut niveau extraites par l'encodeur en une image de sortie reconstruite. Il est composé également de plusieurs blocs, et utilise des couches de convolution transposées pour effectuer l'upsampling des features, rétablissant ainsi leurs dimensions originales. Chaque bloc du décodeur reçoit non seulement les données upsampled du bloc précédent, mais aussi les données de l'encodeur correspondant grâce aux connexions de saut. Ces données concaténées sont ensuite transmises à travers des couches de convolution pour affiner les features upsampled.


## Debruitage des images

On va essayer d'améliorer notre modèle de débruitage d'images qu'on a fait en TP3 en utilisant l'architecture UNet.

## Importation des librairies et téléchargements de MNIST

In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

X_train = train_dataset.data
X_test = test_dataset.data

print(X_train.shape)
print(X_test.shape)

## Exercice 1: Normalisation des images et ajout de bruit
Normaliser les images en [0, 1] et ajouter du bruit sur les images de train et de test.

In [ ]:
X_train_normalized = ...
X_test_normalized = ...
...
X_train_noise = ...
X_test_noise = ...

## Exercice 2: Visualisation des données bruitées

In [ ]:
...

## Exercice 3: Création du modèle


### Encodeur
Voici le schéma de l'encodeur qu'on va créer (avec des ReLU après chaque couche de convolution):


In [ ]:
# Layer (type)                   Output Shape              
# ================================================================
# input_1               (1, 28, 28)
# ________________________________________________________________
# conv2d_1              (32, 28, 28)
# ________________________________________________________________
# maxpooling2d_1        (32, 14, 14)
# ________________________________________________________________
# conv2d_2              (64, 14, 14)
# ________________________________________________________________
# maxpooling2d_2        (64, 7, 7)
# ________________________________________________________________
# conv2d_3 (Conv2D)     (128, 7, 7)
# ________________________________________________________________
# maxpooling2d_3        (128, 3, 3)
# ________________________________________________________________
# conv2d_4              (256, 3, 3)
# ________________________________________________________________
# maxpooling2d_4        (256, 1, 1)
# ________________________________________________________________
# conv2d_5              (256, 1, 1)
# ________________________________________________________________
# outputs               [(32, 28, 28), (64, 14, 14), (128, 7, 7), (256, 3, 3), (256, 1, 1)]

Il faut retourner non seulement la sortie de conv2d_5 mais aussi les sorties de conv2d_1, conv2d_2, conv2d_3 et conv2d_4 pour les utiliser dans le décodeur.
Vous pouvez par exemple mettre les sorties dans une liste et les retourner.

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
	def __init__(self):
		super().__init__()
		...

	def forward(self, x):
		...
		return [...]

### Torchinfo pour vérifier

In [ ]:
from torchinfo import summary

batch_size = 5
encoder = Encoder()
summary(encoder, input_size=(batch_size, 1, 28, 28))

### Décodeur
Pour concatener, regarder la fonction torch.cat: https://pytorch.org/docs/stable/generated/torch.cat.html

Voici le schéma du décodeur qu'on va créer (avec des ReLU après chaque couche de convolution):

In [ ]:
# Layer (type)                   Output Shape
# ================================================================
# input_1               [(32, 28, 28), (64, 14, 14), (128, 7, 7), (256, 3, 3), (256, 1, 1)]
# ________________________________________________________________
# conv2d_transpose_1    (256, 3, 3)
# ________________________________________________________________
# concatenate_1         (512, 3, 3)
# ________________________________________________________________
# conv2d_6              (256, 3, 3)
# ________________________________________________________________
# conv2d_transpose_2    (128, 7, 7)
# ________________________________________________________________
# concatenate_2         (256, 7, 7)
# ________________________________________________________________
# conv2d_7              (128, 7, 7)
# ________________________________________________________________
# conv2d_transpose_3    (64, 14, 14)
# ________________________________________________________________
# concatenate_3         (128, 14, 14)
# ________________________________________________________________
# conv2d_8              (64, 14, 14)
# ________________________________________________________________
# conv2d_transpose_4    (32, 28, 28)
# ________________________________________________________________
# concatenate_4         (64, 28, 28)
# ________________________________________________________________
# conv2d_9              (1, 28, 28)
# ________________________________________________________________


In [ ]:
from typing import List


class Decoder(nn.Module):
	def __init__(self):
		super().__init__()
		...

	def forward(self, *encoder_outputs: List[torch.Tensor]):
		...


### Torchinfo pour vérifier

In [ ]:
from torchinfo import summary

batch_size = 5
decoder = Decoder()
summary(decoder, input_size=[(batch_size, 32, 28, 28), (batch_size, 64, 14, 14), (batch_size, 128, 7, 7), (batch_size, 256, 3, 3), (batch_size, 256, 1, 1)])

### Unet

In [ ]:
class Unet(nn.Module):
	def __init__(self):
		super().__init__()
		self.encoder = Encoder()
		self.decoder = Decoder()

	def forward(self, x):
		outs = self.encoder(x)
		out = self.decoder(*outs)
		return out

In [ ]:
batch_size = 5
unet = Unet()
summary(decoder, input_size=(batch_size, 1, 28, 28))

## Exercice 4: Entraînement du modèle (sur GPU) et visualisation des résultats

In [ ]:
...

In [ ]:
import matplotlib.pyplot as plt
...

## Challenge: Denoising Dirty Documents/Remove noise from printed text

![Dirty Documents](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/TP3_Autoencodeur/denoisingdocument.JPG)

Lien vers le challenge: https://sharing.cs-campus.fr/compete/89